In [1]:
# Use sqlite file for output
import sqlite3
import pandas
con = sqlite3.connect('SO_quality.db')
cur = con.cursor()

In [2]:
cur.execute("drop table if exists data;")
cur.execute(
"create table data( \
    id int, \
    ViewCount int, \
    AnswerCount int, \
    Score int, \
    ViewRatio float, \
    AnswerRatio float, \
    Category varchar \
);")
from xml.dom import minidom
categories =['earthscience', 'physics', 'chemistry', 'astronomy', 'biology']
for category in categories:
    xmldoc = minidom.parse(category + '.stackexchange.com/Posts.xml')
    postlist = xmldoc.getElementsByTagName('row')
    to_db = []
    for post in postlist:
        if int(post.attributes['PostTypeId'].value) is not 1:
            continue
        post_id = int(post.attributes['Id'].value)
        ViewCount = int(post.attributes['ViewCount'].value)
        AnswerCount = int(post.attributes['AnswerCount'].value)
        Score = int(post.attributes['Score'].value)
        ViewRatio = float(Score) / ViewCount
        AnswerRatio = float(Score) / AnswerCount if AnswerCount > 0 else None
        to_db.append((post_id, ViewCount, AnswerCount, Score, ViewRatio, AnswerRatio, category))
    cur.executemany("insert into data (id, ViewCount, AnswerCount, Score, ViewRatio, AnswerRatio, Category ) \
                 VALUES (?, ?, ?, ?, ?, ?, ?);", to_db)

In [4]:
import numpy
ViewRatioTable_NonNegScores = pandas.read_sql('select ViewRatio from data where Score >= 0 AND ViewCount > 100', con)
AnswerRatioTable_NonNegScores = pandas.read_sql('select AnswerRatio from data where Score >= 0 AND AnswerRatio is NOT NULL', con)
Avg_ViewRatioTable_NonNegScores = numpy.average(ViewRatioTable_NonNegScores.values)
Avg_AnswerRatioTable_NonNegScores = numpy.average(AnswerRatioTable_NonNegScores.values)
cur.execute("DROP TABLE IF EXISTS good_bad;")
cur.execute("CREATE TABLE good_bad \
            AS SELECT *, AnswerRatio IS NOT NULL AND AnswerRatio > " + str(Avg_AnswerRatioTable_NonNegScores ) + " as answer_good, \
            AnswerRatio IS NOT NULL AND Score IS 0 as answer_bad, \
            ViewCount > 100 AND ViewRatio > " + str(Avg_ViewRatioTable_NonNegScores / 2) + " as view_good, \
            ViewCount > 100 AND Score IS 0 as view_bad from data;")
print(pandas.read_sql('select answer_good, answer_bad, view_good, view_bad, count(*) from good_bad \
                        GROUP BY answer_good, answer_bad, view_good, view_bad', con))
con.commit()
con.close()

   answer_good  answer_bad  view_good  view_bad  count(*)
0            0           0          0         0     68608
1            0           0          0         1      1964
2            0           0          1         0     28390
3            0           1          0         0     13001
4            0           1          0         1     16827
5            1           0          0         0     13116
6            1           0          1         0     27233


In [6]:
# Make new database for training and testing
con = sqlite3.connect('SO.db')
cur = con.cursor()
cur.execute("DROP TABLE IF EXISTS data;")
cur.execute(
"create table data( \
    id int, \
    ViewCount int, \
    AnswerCount int, \
    Score int, \
    ViewRatio float, \
    AnswerRatio float, \
    Category varchar, \
    Body varchar, \
    Title varchar, \
    view_good, \
    view_bad, \
    answer_good, \
    answer_bad \
);")
from xml.dom import minidom
categories =['earthscience', 'physics', 'chemistry', 'astronomy', 'biology']
for category in categories:
    xmldoc = minidom.parse(category + '.stackexchange.com/Posts.xml')
    postlist = xmldoc.getElementsByTagName('row')
    to_db = []
    for post in postlist:
        if int(post.attributes['PostTypeId'].value) is not 1:
            continue
        post_id = int(post.attributes['Id'].value)
        ViewCount = int(post.attributes['ViewCount'].value)
        AnswerCount = int(post.attributes['AnswerCount'].value)
        Score = int(post.attributes['Score'].value)
        ViewRatio = float(Score) / ViewCount
        AnswerRatio = float(Score) / AnswerCount if AnswerCount > 0 else None
        body = post.attributes['Body'].value
        title = post.attributes['Title'].value
        answer_good = AnswerCount > 0 and AnswerRatio > Avg_AnswerRatioTable_NonNegScores
        answer_bad = AnswerCount > 0 and Score is 0
        view_good = ViewCount > 100 and ViewRatio > Avg_ViewRatioTable_NonNegScores / 2
        view_bad = ViewCount > 100 and Score is 0
        to_db.append((post_id, ViewCount, AnswerCount, Score, ViewRatio, AnswerRatio, category, body, title,
                     answer_good, answer_bad, view_good, view_bad))
    cur.executemany("insert into data (id, ViewCount, AnswerCount, Score, ViewRatio, AnswerRatio, Category, \
                    Body, Title, view_good, view_bad, answer_good, answer_bad) \
                 VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", to_db)
# Attach the quality database and read out all the things, including the good and the bad
# So that thing includes the metrics of data quality. It's a thing. Maybe include that generation part here as well?
# Okay great gonna do that 
# Then I make a new database, just SO.db, that has the quality metrics and all of the text

In [7]:
con.commit()
con.close()